In [115]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import math 
from transformer import GPTLanguageModel as model

In [116]:
df = pd.read_csv("data/BTC-USD_hourly.csv")
df.head()

,unix,date,symbol,open,high,low,close,volume
0,1.675210e+12,31/01/2023 23:00,BTC/USD,23197.91,23197.91,23088.07,23138.09,189986.2721
1,1.675200e+12,31/01/2023 22:00,BTC/USD,22944.25,23303.72,22886.10,23197.91,485198.6068
2,1.675200e+12,31/01/2023 21:00,BTC/USD,23106.60,23128.48,22824.34,22944.25,727566.7846
3,1.675200e+12,31/01/2023 20:00,BTC/USD,23169.34,23169.44,23052.70,23106.60,589078.5118
4,1.675190e+12,31/01/2023 19:00,BTC/USD,23181.09,23190.00,23100.00,23169.34,344376.3959


In [117]:
close = df[::-1].close.to_list()

In [118]:
# Encode price data into tokenized list by using histogram bins 
def encode(ls: list, bins: list):
    tokenized_data = []
    for i in range(len(ls)):
        j = 0
        while j < len(bins) -1:
            if ls[i] < bins[j+1]:
                break
            j += 1 
        tokenized_data.append(j)

    return tokenized_data

In [119]:
# Return a list of the bin ranges in the form of a single list according to a number of bins
# Method to do this is split the data into +/- and split each of these halves into equally 
# populated bins. There will be an equal number of bins below and above 0
def return_binsv2(ls: list, n_bins: int):

    if n_bins % 2 != 0:
        return None
    
    n_bins_half = round(n_bins/2)
    
    ls_postive = [x for x in ls if x >= 0]
    ls_negative = [x for x in ls if x < 0]

    pos_tokens, pos_bins = pd.qcut(ls_postive, q=n_bins_half, labels=False, retbins=True)
    neg_tokens, neg_bins = pd.qcut(ls_negative, q=n_bins_half, labels=False, retbins=True)
    
    return [neg_bins[0]*1.1] + neg_bins.tolist()[1:-1] + [-0.00001, 0.00001] + pos_bins.tolist()[1:-1] + [pos_bins[-1]*1.1]


In [120]:
# Turn a list of numbers into the percentage changes from one element to the next
def perc_change(prices):
    percentage_changes = []
    for i in range(len(prices)-1):
        percentage_changes.append(100*(prices[i+1]-prices[i])/prices[i])

    return percentage_changes

In [121]:
# Turn tokenized data into an approximation of price by selecting the middle of the bin range
def tokens_to_price(tokens, bins):
    price_return = []
    for t in tokens:
        price_return.append((bins[t]+bins[t+1])/2)

    return price_return

In [122]:
def tokenize_price_data(prices):
    percentage_changes = perc_change(prices)
    bins = return_binsv2(percentage_changes, 20)
    tokenized_price_change = encode(percentage_changes, bins)

    return tokenized_price_change, bins

In [123]:
percentage_changes = perc_change(close)
bins = return_binsv2(percentage_changes, 20)
tokenized_price_change = encode(percentage_changes, bins)
print(bins)
print(min(percentage_changes))

[-20.39146034522769, -1.2089352957651582, -0.7386893084496267, -0.5178391600673936, -0.3787674884996244, -0.2792300160066365, -0.20004104795465907, -0.13860762020806144, -0.08519750055345618, -0.037911241079558206, -1e-05, 1e-05, 0.006071434766764648, 0.05509429104717443, 0.1087599206614913, 0.17189742862713367, 0.2458899185133424, 0.345296641205391, 0.4809937888198797, 0.6882881740968038, 1.1141091594897978, 22.15838942307692]
-18.53769122293426


In [124]:
tokens, bins = tokenize_price_data(close)

print(tokens[:20])
print(tokens_to_price(tokens, bins)[:10])

[10, 9, 5, 16, 3, 10, 10, 9, 10, 10, 6, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[0.0, -0.018960620539779104, -0.2396355319806478, 0.2955932798593667, -0.448303324283509, 0.0, 0.0, -0.018960620539779104, 0.0, 0.0]


In [125]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [126]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(tokens)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: [itos[i] for i in l] # decoder: take a list of integers, output a string

In [127]:
# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 45 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 8
n_layer = 8
dropout = 0.2
# ------------

In [128]:
# Train and test splits
data = torch.tensor(encode(tokens), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print(val_data[-100:])

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
start_index = 60400
x_test = torch.asarray(tokens[start_index:start_index+block_size]).reshape((1, block_size))

print(x_test.shape, x_test)

y_pred = decode(m.generate(x_test, max_new_tokens=20)[0].tolist())
y_test = tokens[start_index+block_size:start_index+block_size+21]

print(y_pred)
print(y_test)

tensor([ 2,  6,  3, 16, 17,  5, 12, 14,  7,  6,  8,  5,  9, 13,  8, 11,  6, 14,
         8, 16, 12,  9,  7, 15,  4, 15,  7, 14, 17, 18,  3, 15, 14, 13,  5, 12,
         5, 18, 19,  9, 15, 15,  9,  8, 16,  9, 17, 19,  1, 17,  6,  9, 12,  3,
         7, 16,  9, 13,  7,  6,  3,  0, 15,  2,  9,  7, 19,  3,  9,  5, 14,  0,
         3, 17,  7, 17, 11, 12, 14,  7, 13,  3, 14, 19,  5,  8,  5, 15, 15, 19,
        12,  9,  8, 15, 14,  8,  5,  2, 19,  5])
0.103733 M parameters
step 0: train loss 3.0498, val loss 3.0544
step 500: train loss 2.8254, val loss 2.8485
step 1000: train loss 2.8225, val loss 2.8515
step 1500: train loss 2.8029, val loss 2.8385
step 2000: train loss 2.8075, val loss 2.8425
step 2500: train loss 2.8032, val loss 2.8368
step 3000: train loss 2.7971, val loss 2.8395
step 3500: train loss 2.7920, val loss 2.8406
step 4000: train loss 2.7834, val loss 2.8340
step 4500: train loss 2.7819, val loss 2.8450
step 4999: train loss 2.7768, val loss 2.8410


RuntimeError: shape '[1, 50]' is invalid for input of size 45